# Boosting算法测试

In [3]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import time

# 机器学习模块
import xgboost as xgb

# 回测框架
import backtrader as bt
import backtrader.indicators as btind
import backtrader.feeds as btfeed

# 基础函数
import utilsJ

## 策略主体

In [4]:
class ml(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 1),
        ('p_stake', 100),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.p.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        
        # Initialization
        self.buyprice = 0
        self.sellprice = 0
        self.order = None

        # Alias
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Indicators
        self.bbands = btind.BBands(period=20, devfactor=2)
        self.macdhisto = btind.MACDHisto()
        self.wr = btind.WilliamsR(period=14)
        
        # Machine learning labels
        self.updown = bt.If(self.dataclose(0) > self.dataclose(-1), 1, 0)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price, order.executed.size,
                          self.broker.get_cash()))
                self.buyprice = order.executed.price

            else:
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price, -order.executed.size,
                         self.broker.get_cash()))
                self.sellprice = order.executed.price

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))


    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        
        # 训练集：前30天的数据
        data_train = xgb.DMatrix(pd.DataFrame({'BBands':np.array(self.bollinger.get(ago=-1, size=30)), 
                                               'MACD':np.array(self.macdhisto.get(ago=-1, size=30)),
                                               'wr':np.array(self.wr.get(ago=-1, size=30))}), 
                                 label=np.array(self.updown.get(size=30)))
        #测试集：当天的数据
        data_test = xgb.DMatrix(pd.DataFrame({'BBands':np.array(self.bollinger.get(size=1)), 
                                              'MACD':np.array(self.macdhisto.get(size=1)), 
                                              'wr':np.array(self.wr.get(size=1))}))
        params={'booster':'gbtree',
                'nthread':12,
                'objective': 'rank:pairwise',
                'eval_metric':'auc',
                'seed':0,
                'eta': 0.01,
                'gamma':0.1,
                'min_child_weight':1.1,
                'max_depth':5,
                'lambda':10,
                'subsample':0.7,
                'colsample_bytree':0.7,
                'colsample_bylevel':0.7,
                'tree_method':'exact'
                }
        model = xgb.train(params,data_train)
        print(model.predict(data_test)[0])

### 单股回测测试

In [15]:
stock_code = '002057.SZ'
startdate = dt.datetime(2021, 1, 1)
enddate = dt.datetime(2021,12,31)
    
if __name__ ==  '__main__':

    # Initialization
    cerebro = bt.Cerebro()
    strats = cerebro.addstrategy(ml, printlog=True) 

    # Data
    df = utilsJ.get_stock(stock_code, startdate, enddate)
    data = btfeed.PandasData(dataname=df,fromdate=startdate,todate=enddate)
    cerebro.adddata(data)

    # Initialization
    cerebro.broker = bt.brokers.BackBroker(coc=True)   
    cerebro.broker.setcash(20000)
    #cerebro.broker.setcommission()
    start_value = cerebro.broker.getvalue()
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Execution
    cerebro.run()

    # Final result
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Net Profit: %.2f%%' % ((final_value - start_value) / start_value * 100))

    # Visualization
    cerebro.plot(iplot=False)

Starting Portfolio Value: 20000.00
0.4837914
0.49373007
0.4913369
0.51058906
0.5139871
0.51782155
0.5191905
0.49646908
0.48473603
0.47490206
0.50104487
0.49906504
0.48988417
0.5146203
0.48867747
0.50333935
0.49409157
0.49695104


d:\Anaconda\envs\py38\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


0.49400327
0.5018475
0.48143038
0.48620287
0.4788569
0.4964859
0.48795965
0.49230155
0.4941523
0.50639623
0.5018783
0.48794255
0.50175333
0.5023785
0.50849015
0.51749
0.5049873
0.49527043
0.48642132
0.50733364
0.5009927
0.5036606
0.5130356
0.49603692
0.5034438
0.5105916
0.51126665
0.5133843
0.50133157
0.48659867
0.5015455
0.5028107
0.5156559
0.50003344
0.4906981
0.4830789
0.5021655
0.51187235
0.5148627
0.48409018
0.47922269
0.49884474
0.4964201
0.513952
0.5193462
0.5037922
0.48925084
0.49085528
0.498052
0.5131514
0.5119449
0.4991718
0.49195278
0.48077768
0.47995836
0.49235398
0.5286625
0.5059286
0.5155701
0.52463657
0.5071975
0.49135995
0.5081619
0.51652706
0.52174985
0.52675784
0.52969956
0.52053165
0.49978328
0.4891693
0.49120468
0.4945015
0.49499995
0.48906678
0.50479203
0.5119954
0.5114954
0.50306606
0.49982664
0.51894134
0.50534415
0.4928492
0.48097077
0.47916225
0.4706255
0.4915186
0.50496966
0.51568174
0.4936564
0.50385624
0.499379
0.51735127
0.5266302
0.519397
0.5078576
0.51718